# Predict new locations for UBS wealth management branch offices.

Union Bank of Switzerland (UBS) is looking to apply Machine Learning in predicting most likely new locations/zipcodes be opened with UBS wealth management branches in the USA

### keywords: data wrangling, PCA, under-sampling, cross-validation, cross-prediction, LogisticRegression, Naive Bayes, KNN, ensemble learning 

In [28]:
#Import libraries
import numpy as np
import pandas as pd
import random
import copy
from sklearn.model_selection import KFold, cross_val_score,cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier


# Data wrangling

In [29]:
#Import original dataset
original_dataset=pd.read_csv('UBS_original_data.csv')
dataset=copy.deepcopy(original_dataset)
dataset.head()

,ZipCode,Population,HouseholdsPerZipCode,WhitePopulation,BlackPopulation,HispanicPopulation,AsianPopulation,HawaiianPopulation,IndianPopulation,OtherPopulation,...,DeliveryTotal,PopulationEstimate,LandArea,WaterArea,BoxCount,SFDU,MFDU,CityDeliveryIndicator,MedicareCBSAType,MarketRatingAreaID
0,501,0,0,0,0,0,0,0,0,0,...,1,0,0.000,0.000,0,0,0,N,Metro,8
1,501,0,0,0,0,0,0,0,0,0,...,1,0,0.000,0.000,0,0,0,N,Metro,8
2,544,0,0,0,0,0,0,0,0,0,...,0,0,0.000,0.000,0,0,0,N,Metro,8
3,544,0,0,0,0,0,0,0,0,0,...,0,0,0.000,0.000,0,0,0,N,Metro,8
4,601,18570,6525,17479,663,18486,7,10,113,558,...,5074,11342,64.348,0.309,831,2376,1206,Y,Micro,1


In [30]:
#Remove the rows where the population are 0
dataset = dataset[dataset.Population != 0]
dataset.head()

,ZipCode,Population,HouseholdsPerZipCode,WhitePopulation,BlackPopulation,HispanicPopulation,AsianPopulation,HawaiianPopulation,IndianPopulation,OtherPopulation,...,DeliveryTotal,PopulationEstimate,LandArea,WaterArea,BoxCount,SFDU,MFDU,CityDeliveryIndicator,MedicareCBSAType,MarketRatingAreaID
4,601,18570,6525,17479,663,18486,7,10,113,558,...,5074,11342,64.348,0.309,831,2376,1206,Y,Micro,1
5,601,18570,6525,17479,663,18486,7,10,113,558,...,5074,11342,64.348,0.309,831,2376,1206,Y,Micro,1
6,601,18570,6525,17479,663,18486,7,10,113,558,...,5074,11342,64.348,0.309,831,2376,1206,Y,Micro,1
7,601,18570,6525,17479,663,18486,7,10,113,558,...,5074,11342,64.348,0.309,831,2376,1206,Y,Micro,1
8,602,41520,15002,36828,2860,41265,42,32,291,2634,...,11165,24000,30.613,1.717,1502,5420,821,Y,Metro,1


In [31]:
#Reset the row index for later use
dataset.index=range(len(dataset))
#Replace the empty cell with the NaN
for j in range(dataset.columns.size):
    dataset.iloc[:,j].replace(r'\s+', np.nan, regex=True, inplace=True)

In [32]:
#Checking for missing values(NaN) in each feature
feature_nan_value=dataset.isnull().sum()
print(feature_nan_value)

ZipCode                            0
Population                         0
HouseholdsPerZipCode               0
WhitePopulation                    0
BlackPopulation                    0
HispanicPopulation                 0
AsianPopulation                    0
HawaiianPopulation                 0
IndianPopulation                   0
OtherPopulation                    0
MalePopulation                     0
FemalePopulation                   0
PersonsPerHousehold                0
AverageHouseValue                  0
IncomePerHousehold                 0
MedianAge                          0
MedianAgeMale                      0
MedianAgeFemale                    0
CityType                           0
NumberOfBusinesses                 0
NumberOfEmployees                  0
BusinessFirstQuarterPayroll        0
BusinessAnnualPayroll              0
BusinessEmploymentFlag         62853
GrowthRank                         0
GrowingCountiesA                   0
GrowingCountiesB                   0
G

In [33]:
#Drop those features with over 50% of the values being NaN 
dataset.drop(['BusinessEmploymentFlag'],axis=1,inplace=True)

In [34]:
#Fill the nan with its next value for the feature "MedicareCBSAType" 
dataset.MedicareCBSAType.fillna(method='bfill',inplace=True)

In [35]:
#Encode categorical features with dummy variables
dataset=pd.get_dummies(dataset)  

In [36]:
#Group by zip code, and select max value of each group
dataset=dataset.groupby('ZipCode',as_index=False).max()

In [37]:
#Input label/target data
#the zipcodes already opened with branches are labeled with "1", otherwise "0"
dataset_label=pd.read_csv('UBS_data_label.csv')
dataset_label.head()

,ZipCode,UBS_Open_Branch
0,1144,1
1,1608,1
2,1945,1
3,1960,1
4,2109,1


In [38]:
#Final clean data with the label
#Final_dataset=pd.merge(dataset,dataset_label.drop(['UBS_Branches'],axis=1), on='ZipCode',how='left')
Final_dataset=pd.merge(dataset,dataset_label, on='ZipCode',how='left')
Final_dataset.UBS_Open_Branch.fillna(0,inplace=True)
Final_dataset.to_csv('UBS_clean_data.csv',index=False)

In [39]:
# Import clean dataset
original_dataset = pd.read_csv('UBS_clean_data.csv')
original_dataset.head()

,ZipCode,Population,HouseholdsPerZipCode,WhitePopulation,BlackPopulation,HispanicPopulation,AsianPopulation,HawaiianPopulation,IndianPopulation,OtherPopulation,...,CityType_C,CityType_N,CityType_P,CityType_U,CityType_Z,CityDeliveryIndicator_N,CityDeliveryIndicator_Y,MedicareCBSAType_Metro,MedicareCBSAType_Micro,UBS_Open_Branch
0,601,18570,6525,17479,663,18486,7,10,113,558,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,602,41520,15002,36828,2860,41265,42,32,291,2634,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
2,603,54689,21161,46501,5042,53877,135,35,313,4177,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
3,606,6615,2404,5979,371,6575,3,9,35,323,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,610,29016,10836,24510,2654,28789,57,31,200,2494,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0


## Principal Component Analysis (PCA)

In [40]:
# Import StandardScaler
from sklearn.preprocessing import StandardScaler 

In [41]:
# Scale the features and set the values to a new variable
scaler = StandardScaler() 
original_dataset_features = scaler.fit_transform(original_dataset.drop(['ZipCode','UBS_Open_Branch'],axis=1))

C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [42]:
# Import PCA class
from sklearn.decomposition import PCA

In [43]:
#Find the suitable number of PCA components 
pca = PCA(random_state=7)
pca.fit(original_dataset_features)
#Get explained variance ratios from PCA using all features
exp_variance = pca.fit(original_dataset_features).explained_variance_ratio_

In [17]:
# Calculate the cumulative explained variance
cum_exp_variance = np.cumsum(exp_variance)

In [18]:
#Determine the number of components by 90% 
#of the cumulative explained variance  
n_components = len(cum_exp_variance[cum_exp_variance<0.9])
print(n_components)

19


In [19]:
#Perform PCA with the chosen number of components and project data onto components
pca = PCA(n_components, random_state=10)
pca.fit(original_dataset_features)
PCA_dataset_features= pca.transform(original_dataset_features)
PCA_dataset_features=pd.DataFrame(PCA_dataset_features)

In [20]:
#Add label and zip code index
PCA_dataset=PCA_dataset_features
PCA_dataset['UBS_Open_Branch']=original_dataset['UBS_Open_Branch']
PCA_dataset['ZipCode']=original_dataset['ZipCode']
#Add prediction coloumn
PCA_dataset['y_pred_prob']=0.0

In [21]:
#Number of records in class 0 and class 1
print('The size of class 1: {}'.format(len(PCA_dataset[PCA_dataset.UBS_Open_Branch == 0])))
print('The size of class 0: {}'.format(len(PCA_dataset[PCA_dataset.UBS_Open_Branch != 0]))) 

The size of class 1: 32695
The size of class 0: 264


## Binary classification problem on a highly imbalanced dataset

In [22]:
# Minority dataset
minority_dataset=PCA_dataset[PCA_dataset.UBS_Open_Branch != 0]
# Majority dataset
majority_dataset = PCA_dataset[PCA_dataset.UBS_Open_Branch == 0]

In [23]:
#Shuffle the indeces of Majority dataset
majority_index=list(majority_dataset.index)
random.seed(2337)
random.shuffle(majority_index)

## Undersampling & cross-prediction 

For this specific problem, the information of zipcodes in majority class are requied for model training, on the other hand, any zipcode in majority class is possbile to be opened with a new branch.
Therefore, regular model training and prediction process lead to "data leakage", which refers to a mistake made by the creator of a machine learning model, in which they accidentally share information between the test and training data-sets, memorizes the training set data, and is easily able to correctly output the labels/values for those test data-set examples.
In order to avoid "data leakage", we employ an undersampling & cross-prediction scheme.

Divide the majority dataset into m=122 subsets, each subset size is n=268, we randomly select one subset of majority dataset, and combine it with the minority set as a train set to train the model with cross-validation, the cross-validation score is recored as the weight which is used in the final stage. 
Keep the remaining part of the majority dataset as the test set, predict the test set with the trained model.
<img src="cross_prediction2-1.png">
Repeat above process until all the m=122 subsets of majority dataset have been employed for model training. 
Finally calculate the weighted average of the predictions for each zipcode, 
those zipcodes with the top predictions are the ones to be opend wth the new branches most likely.

In [24]:
#Parameters initialization

simulation_rounds=122

#Size of each subset in majority set
subsetsize=268

#Indeces of elements in each subset of majority set
majority_subset_index=[]

#Cross-validation score initialization
Model_performance=pd.DataFrame(np.zeros((122,1)),columns=['Accuracy'])

#Obtain indeces of elements in each subset of majority set
for i in range(122):
    majority_subset_index.append(majority_index[i*subsetsize:(i+1)*subsetsize])
    
#Initialize the final weighted prediction of each zipcode
Final_All_ZipCode_prediction=PCA_dataset.loc[:,['ZipCode','y_pred_prob']]

#Initialize the prediction of each zipcode in a single round cross-prediction
temp_single_round_prediction=PCA_dataset.loc[:,['ZipCode','y_pred_prob']] 

In [25]:
#Train model with K-Fold cross validation
kf = KFold(n_splits=10,random_state=10)

#Classification model ensembling

#Logistic regression
logreg= LogisticRegression(random_state=10,solver='liblinear')#random_state=10
#Naive bayes
NB = GaussianNB()
# Decision tree
#tree= DecisionTreeClassifier(criterion = 'entropy', random_state = 10) 
#KNN
KNN = KNeighborsClassifier(n_neighbors =15, metric = 'minkowski', p = 2)

In [26]:
#Cross prediction
NB_count=0
for j in range(simulation_rounds):
    
    #A subset of majoriry data set is combined with the minority set to form the train set
    majority_datasubset=majority_dataset.loc[majority_subset_index[j],:]
    dataset_train=majority_datasubset.append(minority_dataset,ignore_index=False)    
    X_train= dataset_train.iloc[:,1:-3].values ##features
    y_train = dataset_train.UBS_Open_Branch.values##labels
      
    
    #Train models using KFold cv
    #logit_score =cross_val_score(logreg, X_train, y_train, cv=kf,scoring='accuracy')
    #NB_score = cross_val_score(NB, X_train, y_train, cv=kf,scoring='accuracy')
    logit_score = cross_validate(logreg, X_train, y_train,  scoring='accuracy', cv=kf)
    NB_score=cross_validate(NB, X_train, y_train,  scoring='accuracy', cv=kf)
    #tree_score=cross_validate(tree, X_train, y_train,  scoring='accuracy', cv=kf)
    KNN_score=cross_validate(KNN, X_train, y_train,  scoring='accuracy', cv=kf)
    
    #Select the average accuracy score of different models
    cross_validation_score = (np.mean(NB_score['test_score'])+np.mean(logit_score['test_score'])+np.mean(KNN_score['test_score']))/3
    
    #Predict the test set
    #The remaining part of the majority dataset which is not used in training process
    #is the test set   
    X_test=majority_dataset.drop(majority_subset_index[j],axis=0).iloc[:,1:-3].values
    X_test_ZipCode=majority_dataset.drop(majority_subset_index[j],axis=0).loc[:,['ZipCode','y_pred_prob']]
     

    #Predict the test set with different models   

    NB.fit(X_train, y_train)           
    y_probas_NB= NB.predict_proba(X_test)

    logreg.fit(X_train, y_train) 
    y_probas_logit= logreg.predict_proba(X_test)
    
    #tree.fit(X_train, y_train) 
    #y_probas_tree= tree.predict_proba(X_test)
    
    KNN.fit(X_train, y_train) 
    y_probas_KNN= KNN.predict_proba(X_test)
    
    # Get the average predicted probability of text set with different models
    y_probas=(y_probas_NB+y_probas_logit+y_probas_KNN)/3
    
    X_test_ZipCode['y_pred_prob']=y_probas[:,1]
    
    #Merge the predictions on Zipcode
    single_round_predictions=pd.merge(temp_single_round_prediction,X_test_ZipCode, on='ZipCode',how='left')
    
    #Replace nan with 0
    single_round_predictions.iloc[:,-1].fillna(0,inplace=True)
    
    
    #Weighted averaging the prediction probability with the 
    #Cross validation score from each model
    Final_All_ZipCode_prediction['y_pred_prob']+=single_round_predictions.iloc[:,-1]*cross_validation_score 
    Model_performance.iloc[j,:]=cross_validation_score
    print('Cross-validation for {}-th subset using LogRg: train_score mean is {}, test_score mean is {}'.format(j,np.mean(logit_score['train_score']),np.mean(logit_score['test_score'])))
    print('Cross-validation for {}-th subset using    NB: train_score mean is {}, test_score mean is {}'.format(j,np.mean(NB_score['train_score']),np.mean(NB_score['test_score'])))
    #print('Cross-validation for {}-th subset using tree: train_score mean is {}, test_score mean is {}'.format(j,np.mean(tree_score['train_score']),np.mean(tree_score['test_score'])))
    print('Cross-validation for {}-th subset using   KNN: train_score mean is {}, test_score mean is {}'.format(j,np.mean(KNN_score['train_score']),np.mean(KNN_score['test_score'])))

C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


Cross-validation for 0-th subset using LogRg: train_score mean is 0.8926455044941954, test_score mean is 0.836408106219427
Cross-validation for 0-th subset using    NB: train_score mean is 0.7456058210532752, test_score mean is 0.689203354297694
Cross-validation for 0-th subset using   KNN: train_score mean is 0.9012089342336284, test_score mean is 0.8365478686233402


C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


Cross-validation for 1-th subset using LogRg: train_score mean is 0.898075663210489, test_score mean is 0.8494060097833683
Cross-validation for 1-th subset using    NB: train_score mean is 0.7529166411893676, test_score mean is 0.7154088050314465
Cross-validation for 1-th subset using   KNN: train_score mean is 0.8978686419580543, test_score mean is 0.8288259958071279


C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


...

C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


Cross-validation for 119-th subset using LogRg: train_score mean is 0.8859640464356531, test_score mean is 0.8137316561844864
Cross-validation for 119-th subset using    NB: train_score mean is 0.7788235602414374, test_score mean is 0.7100279524807827
Cross-validation for 119-th subset using   KNN: train_score mean is 0.8945252924066004, test_score mean is 0.8174703004891685


C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


Cross-validation for 120-th subset using LogRg: train_score mean is 0.9101916475222964, test_score mean is 0.8550663871418589
Cross-validation for 120-th subset using    NB: train_score mean is 0.7817380176623194, test_score mean is 0.7419986023759609
Cross-validation for 120-th subset using   KNN: train_score mean is 0.9122819507167129, test_score mean is 0.8570929419986024
Cross-validation for 121-th subset using LogRg: train_score mean is 0.8821943282194328, test_score mean is 0.8117051013277429
Cross-validation for 121-th subset using    NB: train_score mean is 0.7225344947062797, test_score mean is 0.6737945492662474
Cross-validation for 121-th subset using   KNN: train_score mean is 0.8821930124645843, test_score mean is 0.8174004192872119


C:\Users\cy235\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [27]:
#Rank the predicted probability
Final_All_ZipCode_prediction.sort_values('y_pred_prob',inplace=True,ascending=False)
Final_All_ZipCode_prediction['y_pred_prob']=Final_All_ZipCode_prediction['y_pred_prob']/Model_performance.iloc[:,0].sum()
print(Final_All_ZipCode_prediction.head(20).to_string(index=False))

ZipCode  y_pred_prob
  10017     0.988455
  10036     0.988185
  60611     0.987978
  43215     0.987709
  92121     0.987674
  89109     0.987522
  94107     0.987456
  94103     0.987425
  15222     0.987300
  98101     0.987243
  10018     0.987083
  77030     0.986990
  80112     0.986934
  94111     0.986803
  60601     0.986781
   2210     0.986490
  10001     0.986470
   2110     0.986403
  10016     0.986232
  30339     0.986153
